In [402]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    # Install xvfb and our launcher script for it
    !apt-get install -y xvfb
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/xvfb -O ../xvfb

    # Download dependencies from Github
    !wget https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week06_policy_based/atari_wrappers.py
    !wget https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week06_policy_based/env_batch.py
    !wget https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week06_policy_based/runners.py
    
    # Update the gym environment to be compatible with the Atari environment
    !pip install gym[atari,accept-rom-license]

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

# Implementing Advantage-Actor Critic (A2C)

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel.

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscale, take max between frames, skip frames and stack them together) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function. Note that if you are using
PyTorch and not using `tensorboardX` you will need to implement a wrapper that will log **raw** total rewards that the *unwrapped* environment returns and redefine the implemention of `nature_dqn_env` function here.



In [403]:
import numpy as np
from atari_wrappers import nature_dqn_env, NumpySummaries


env_name = 'SpaceInvadersNoFrameskip-v4'
env = nature_dqn_env(env_name, nenvs=8)
obs = env.reset()
assert obs.shape == (8, 84, 84, 4)
assert obs.dtype == np.uint8

/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers

Next, we will need to implement a model that predicts logits and values. It is suggested that you use the same model as in [Nature DQN paper](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) with a modification that instead of having a single output layer, it will have two output layers taking as input the output of the last hidden layer. **Note** that this model is different from the model you used in homework where you implemented DQN. You can use your favorite deep learning framework here. We suggest that you use orthogonal initialization with parameter $\sqrt{2}$ for kernels and initialize biases with zeros.

In [404]:
# import tensorflow as torch
import torch
import torch.nn as nn

# Reference: https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf

def ortho_init(layer):
  if isinstance(layer, nn.Conv2d):
    nn.init.orthogonal_(layer.weight,np.sqrt(2))
    layer.bias.data.fill_(0.01)

class Network(nn.Module):
  def __init__(self, n_actions):
    super().__init__()
    self.n_actions = n_actions
    self.sub_model = nn.Sequential(
        nn.Conv2d(4, 32, 8, 4),
        nn.ReLU(),
        nn.Conv2d(32, 64, 4, 2),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3, 1),
        nn.Flatten(),
        nn.Linear(3136, 512), # 64 outputs with height = width = 7 (Formula: (h_out = (h_in - kernel_size)/stride + 1) given padding = 0)
        nn.ReLU()
    )
    self.sub_model.apply(ortho_init)
  def forward(self, s):
    value_model = nn.Sequential(
        nn.Linear(512, 128),
        nn.Sigmoid(),
        nn.Linear(128, 1)
    )
    logits_model = nn.Linear(512, self.n_actions)
    s = torch.as_tensor(s.transpose(0, 3, 1, 2), dtype=torch.float32)
    s = self.sub_model(s)
    state_value = value_model(s)
    policy_logits = logits_model(s)
    return state_value, policy_logits

You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a dictionary of all the arrays that are needed to interact with an environment and train the model.
 Note that actions must be an `np.ndarray` while the other
tensors need to have the type determined by your deep learning framework.

In [405]:
class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        # Implement a policy by calling the model, sampling actions and computing their log probs.
        # Should return a dict containing keys ['actions', 'logits', 'log_probs', 'values'].
        values, logits = self.model(inputs)
        log_softmax = nn.LogSoftmax(dim = 1)
        softmax = nn.Softmax(dim = 1)
        log_probs = log_softmax(logits)
        probs = softmax(logits).detach().numpy()
        actions = [np.random.choice(np.arange(self.model.n_actions), p = probs[i]) for i in range(len(probs))]        
        return {'actions': actions, 'logits': logits, 'log_probs': log_probs, 'values': values}


Next will pass the environment and policy to a runner that collects partial trajectories from the environment.
The class that does is is already implemented for you.

In [406]:
from runners import EnvRunner

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys

* 'observations'
* 'rewards'
* 'resets'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment. This list has length $T$ that is size of partial trajectory. Partial trajectory for given moment `t` is part of `ComputeValueTargets.__call__` input argument `trajectory` from moment `t` to the end (i.e. it's different at each iteration in the algorithm).

To train the part of the model that predicts state values you will need to compute the value targets.
Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected.
Thus, we can implement and use `ComputeValueTargets` callable.
The formula for the value targets is simple:

$$
\hat v(s_t) = \left( \sum_{t'=0}^{T - 1} \gamma^{t'}r_{t+t'} \right) + \gamma^T \hat{v}(s_{t+T}),
$$

In implementation, however, do not forget to use
`trajectory['resets']` flags to check if you need to add the value targets at the next step when
computing value targets for the current step. You can access `trajectory['state']['latest_observation']`
to get last observations in partial trajectory &mdash; $s_{t+T}$.

In [407]:
class ComputeValueTargets:
    def __init__(self, policy, gamma=0.99):
        self.policy = policy
        self.gamma = gamma

    def __call__(self, trajectory):
        """Compute value targets for a given partial trajectory."""
        # This method should modify trajectory inplace by adding
        # an item with key 'value_targets' to it.
        rewards = trajectory['rewards']
        resets = trajectory['resets']
        n = len(rewards)
        last_value, _ = self.policy.model(trajectory['state']['latest_observation'])# Throw logits and leave values
        last_value = last_value[:,0].detach().numpy() # Problematic Shape


After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `env_steps * nenvs`, i.e. you essentially need
to flatten the first two dimensions.

In [408]:
class MergeTimeBatch:
    """ Merges first two axes typically representing time and env batch. """
    def __call__(self, trajectory):
        # Modify trajectory inplace.


In [409]:
model = Network(env.action_space.n)
policy = Policy(model)
runner = EnvRunner(
    env, policy, nsteps=5,
    transforms=[
        ComputeValueTargets(policy),
        MergeTimeBatch(),
    ])

Now is the time to implement the advantage actor critic algorithm itself. You can look into your lecture,
[Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and [lecture](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) by Sergey Levine.

In [410]:
def to_one_hot(y_tensor, ndims):
    """ helper: take an integer vector and convert it to 1-hot matrix. """
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [411]:
class A2C:
    def __init__(self,
                 policy,
                 optimizer,
                 value_loss_coef=0.25,
                 entropy_coef=0.01,
                 max_grad_norm=0.5):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    def policy_loss(self, trajectory):
        # You will need to compute advantages here.
        n_actions = self.policy.model.n_actions
        policy, values = self.policy.model(trajectory['observations'])
        log_probs = trajectory['log_probs']
        probs = trajectory['probs']
        actions = trajectory['actions']
        rewards = trajectory['rewards']
        entropy = - (probs * log_probs).mean()
        value_targets = ComputeValueTargets(trajectory)
        log_probs_for_actions = torch.sum( 
            log_probs * to_one_hot(actions, n_actions), dim=1)
        p_loss = -(log_probs_for_actions * (value_targets - values) + self.entropy_coef * entropy).mean()
        return p_loss        

    def value_loss(self, trajectory):
        value_targets = trajectory['value_targets']
        values = self.policy.model(trajectory['observations'])[1]
        return torch.mean((value_targets - values.detach()) ** 2)

    def loss(self, trajectory):
        return self.policy_loss(self, trajectory) + self.value_loss(self, trajectory) * self.value_loss_coef

    def step(self, trajectory):
        self.optimizer.zero_grad()
        loss = self.loss(trajectory)
        loss.backward()
        self.optimizer.step()

Now you can train your model. With reasonable hyperparameters training on a single GTX1080 for 10 million steps across all batched environments (which translates to about 5 hours of wall clock time)
it should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last
episodes in each environment in the batch) of about 600. You should plot this quantity with respect to
`runner.step_var` &mdash; the number of interactions with all environments. It is highly
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages
* A2C loss

For optimization we suggest you use RMSProp with learning rate starting from 7e-4 and linearly decayed to 0, smoothing constant (alpha in PyTorch and decay in TensorFlow) equal to 0.99 and epsilon equal to 1e-5.

In [412]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

a2c = A2C(
    policy,
    optimizer    
)

for i in range(int(1e7)):
    trajectory = runner.get_next()
    a2c.step(trajectory)

Call
[array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False])]
[array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False])]
[array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False])]
[array([False, Fa

AttributeError: ignored

### Target networks?

You may recall a technique called "target networks" we used a few weeks ago when we trained a DQN agent to play Atari Breakout and wonder why we have not suggested using them here. The answer is that this is more historical than practical.

While the "chasing the target" problem is still present in actor-critic value estimation and target networks do show up in follow-up papers, the original A3C/A2C papers do not mention them and do not explain this omission.

The hypothesis why this may not be a big deal (compared to Q-learning) goes like this. An A3C/A2C agent selects actions based on policy, not an epsilon greedy exploration function, for which the argmax can change drastically due to tiny errors in function approximation. Therefore, errors in the value target caused by target chasing will cause less damage.

Also, the actor-critic gradient relies on the advantage function $A(s_t, a_t) = Q(s_t, a_t) - V(s_t)$. Compare this to the $Q$-function $Q(s_t, a_t) = r(s_t, a_t) + \gamma \cdot \mathbb{E}_{s_{t+1} \mid s_t, a_t} V(s_{t+1})$ used in Q-learning and SARSA: we would expect that any bias in $V$-function approximation will be carried over from $V(s_{t+1})$ to $V(s_t)$ by gradient updates. However, in the formula for the advantage function the two approximations ($Q$-function and $V$-function) come with opposite signs, and thus the errors will cancel out.

The last reason may be computational. Authors were concerned to beat existent algorithms in the wall-clock learning time, and any overhead of parameter copying (target network update) counted against this goal.